<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/Sistem_rekomendasi_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [6]:
# pip install --upgrade pandas

# **Loading Data**

In [7]:
# pip install kagglehub

In [8]:
import kagglehub

path = kagglehub.dataset_download("artermiloff/steam-games-dataset")
print("Path to dataset files:", path)

import os

# Cek file yang tersedia dalam dataset
files = os.listdir(path)
print("Files in dataset:", files)

# Sesuaikan dengan file yang ada - corrected file name
dataset_path = os.path.join(path, "games_march2025_cleaned.csv")
df = pd.read_csv(dataset_path)

# Tampilkan 5 baris pertama
df.head(10)

Path to dataset files: /kaggle/input/steam-games-dataset
Files in dataset: ['games_may2024_cleaned.csv', 'games_may2024_full.csv', 'games_march2025_full.csv', 'games_march2025_cleaned.csv']


,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608
5,440,Team Fortress 2,2007-10-10,0,0.00,1,'The most fun you can have online' - PC Gamer ...,'The most fun you can have online' - PC Gamer ...,Nine distinct classes provide a broad range of...,NaN,...,0,0,0,0,50817,"{'Free to Play': 62868, 'Hero Shooter': 61020,...",89,1146642,93,8172
6,105600,Terraria,2011-05-16,0,9.99,2,"Dig, Fight, Explore, Build: The very world is ...","Dig, Fight, Explore, Build: The very world is ...","Dig, fight, explore, build! Nothing is impossi...",NaN,...,0,0,0,0,30516,"{'Open World Survival Craft': 16365, 'Sandbox'...",97,1102434,96,8860
7,252490,Rust,2018-02-08,0,39.99,4,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,"“Rust is one of the cruelest games on Steam, a...",...,836,3675,231,0,200902,"{'Survival': 18592, 'Crafting': 11822, 'Multip...",87,993856,87,14960
8,4000,Garry's Mod,2006-11-29,0,5.99,0,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,“It brings back the silly glee of unbridled ex...,...,100,1173,136,40,32384,"{'Sandbox': 18706, 'Moddable': 14479, 'Multipl...",96,985010,96,8277
9,1172470,Apex Legends™,2020-11-04,0,0.00,0,Apex Legends: Takeover About the Game Conquer ...,"Conquer with character in Apex Legends, a free...","Apex Legends is the award-winning, free-to-pla...",“The champion of Battle Royales.” 9/10 – GameS...,...,548,667,211,0,151844,"{'Free to Play': 2170, 'Battle Royale': 1483, ...",67,983230,53,7054


In [9]:
df = df[['name', 'release_date', 'detailed_description', 'tags']]

In [10]:
df['release_year'] = pd.to_datetime(df['release_date']).dt.year

In [11]:
df.isnull().sum()/100

,0
name,0.00
release_date,0.00
detailed_description,1.97
tags,0.00
release_year,0.00


In [12]:
df.dropna(inplace=True)
df.shape

(89421, 5)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89421 entries, 0 to 89617
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  89421 non-null  object
 1   release_date          89421 non-null  object
 2   detailed_description  89421 non-null  object
 3   tags                  89421 non-null  object
 4   release_year          89421 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 3.8+ MB


In [14]:
df = df[df['release_year'] > 2010].reset_index(drop=True)

In [15]:
df['content'] = df['detailed_description'] + ' ' + df['tags']

In [16]:
df = df[['name', 'release_year', 'content']]

In [17]:
df_sample = df.sample(n=5000, random_state=40, ).reset_index(drop=True)
df_sample.shape

(5000, 3)

In [18]:
pip install langdetect

In [19]:
from langdetect import detect
df_sample['language'] = df_sample['content'].apply(lambda x: detect(x))
df_sample = df_sample[df_sample['language'] == 'en']

In [20]:
df_sample

,name,release_year,content,language
0,Always The Same Blue Sky...,2015,🌟 The truly epic sequel 'Iridescence' is out n...,en
1,Card of spirits,2016,"Game rule Every turn, the player of that turn ...",en
2,The Lonesome Fog,2016,Might and Delight invites you to step into an ...,en
3,Sad Robot Lilly,2024,"She is a sex robot, who has been abandoned bec...",en
4,Pill Puzzle,2021,"Always paying attention to the color, type and...",en
...,...,...,...,...
4995,Lumen: World of Color,2024,In Lumen you take on the role of a druid seeki...,en
4996,Triptych Book I: Blockhead,2025,Following in the footsteps of great traditiona...,en
4997,Cloud Jump,2023,Take to the clouds in this single-player 3D pr...,en
4998,Hidden Object - Sweet Home,2017,Can you find all the hidden objects in your ho...,en


In [21]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4915 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          4915 non-null   object
 1   release_year  4915 non-null   int32 
 2   content       4915 non-null   object
 3   language      4915 non-null   object
dtypes: int32(1), object(3)
memory usage: 172.8+ KB


# **Preprocessing**

In [22]:
import re

In [23]:
# clean_spcl = re.compile('[/(){}\|@,;]')
# clean_symbol = re.compile('[^0-9a-z #+_]')
# sastrawi = StopWordRemoverFactory()
# stopworda = sastrawi.get_stop_words()
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# def clean_text(text):
#     text = text.lower()
#     text = clean_spcl.sub(' ', text)
#     text = clean_symbol.sub('', text)
#     text = stemmer.stem(text)
#     text = ' '.join(word for word in text.split() if word not in stopworda)
#     return text

In [24]:
import ast
def convert_text(text):
    try:
        teks = ast.literal_eval(text)
        return " ".join([word.lower() for word in teks]) # Convert to lowercase here
    except (ValueError, SyntaxError):
        if isinstance(text, str):
          return " ".join([word.lower() for word in text.split()]) # Handle string cases
        return text

df_sample['content'] = df_sample['content'].apply(convert_text)

<unknown>:1: SyntaxWarning: invalid decimal literal


In [25]:
def cleaning_text(text) :
    text = re.sub(r'#\w+', '', text) #remove hashtag
    text = re.sub(r'\b\w*\d\w*\b', '', text) #remove words with numbers in the middle
    text = re.sub(r'@\S+', '', text) #remove mentions (@username)
    text = re.sub(r'https?://\S+|www\.\S+', '', text) #remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) #remove special characters
    return text

df_sample['clean_content'] = df_sample['content'].apply(cleaning_text)

In [26]:
# from langdetect import detect
# df_sample['language'] = df_sample['clean_content'].apply(lambda x: detect(x))
# df_sample = df_sample[df_sample['language'] == 'en']

In [27]:
nltk.download('punkt_tab')
df_sample['tokenized'] = df_sample['clean_content'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [28]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
df_sample['stopword'] = df_sample['tokenized'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# from stemmer.porter2 import Porter2Stemmer
# stemmer = Porter2Stemmer()
# factory = StemmerFactory()
# porter = PorterStemmer() # untuk stemming kata tertentu dalam bahasa lain
# stemmer = factory.create_stemmer()

# kata_khusus = ['processing']

# kalimat1_stem = []
# for word in kalimat1:
#     if word in kata_khusus:
#         stemmed = porter.stem(word)
#     else:
#         stemmed = stemmer.stem(word)
#     kalimat1_stem.append(stemmed)

# print(kalimat1_stem)

# df_sample['stemming'] = df_sample['stopword'].apply(lambda x: [stemmer.stem(word) for word in x])

In [30]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:
type(df_sample['stopword'].iloc[0])

list

In [32]:
lematizer = WordNetLemmatizer()
df_sample['lematized'] = df_sample['stopword'].apply(lambda x: [lematizer.lemmatize(word) for word in x]).apply(lambda x: " ".join(x))

In [33]:
df_sample

,name,release_year,content,language,clean_content,tokenized,stopword,lematized
0,Always The Same Blue Sky...,2015,🌟 the truly epic sequel 'iridescence' is out n...,en,the truly epic sequel iridescence is out now ...,"[the, truly, epic, sequel, iridescence, is, ou...","[truly, epic, sequel, iridescence, always, blu...",truly epic sequel iridescence always blue sky ...
1,Card of spirits,2016,"game rule every turn, the player of that turn ...",en,game rule every turn the player of that turn m...,"[game, rule, every, turn, the, player, of, tha...","[game, rule, every, turn, player, turn, must, ...",game rule every turn player turn must play car...
2,The Lonesome Fog,2016,might and delight invites you to step into an ...,en,might and delight invites you to step into an ...,"[might, and, delight, invites, you, to, step, ...","[might, delight, invites, step, interactive, d...",might delight invite step interactive diorama ...
3,Sad Robot Lilly,2024,"she is a sex robot, who has been abandoned bec...",en,she is a sex robot who has been abandoned beca...,"[she, is, a, sex, robot, who, has, been, aband...","[sex, robot, abandoned, humans, verge, extinct...",sex robot abandoned human verge extinct wake o...
4,Pill Puzzle,2021,"always paying attention to the color, type and...",en,always paying attention to the color type and ...,"[always, paying, attention, to, the, color, ty...","[always, paying, attention, color, type, posit...",always paying attention color type position ta...
...,...,...,...,...,...,...,...,...
4995,Lumen: World of Color,2024,in lumen you take on the role of a druid seeki...,en,in lumen you take on the role of a druid seeki...,"[in, lumen, you, take, on, the, role, of, a, d...","[lumen, take, role, druid, seeking, cleanse, a...",lumen take role druid seeking cleanse ancient ...
4996,Triptych Book I: Blockhead,2025,following in the footsteps of great traditiona...,en,following in the footsteps of great traditiona...,"[following, in, the, footsteps, of, great, tra...","[following, footsteps, great, traditional, adv...",following footstep great traditional adventure...
4997,Cloud Jump,2023,take to the clouds in this single-player 3d pr...,en,take to the clouds in this singleplayer preci...,"[take, to, the, clouds, in, this, singleplayer...","[take, clouds, singleplayer, precision, platfo...",take cloud singleplayer precision platformer j...
4998,Hidden Object - Sweet Home,2017,can you find all the hidden objects in your ho...,en,can you find all the hidden objects in your ho...,"[can, you, find, all, the, hidden, objects, in...","[find, hidden, objects, home, help, beautiful,...",find hidden object home help beautiful woman f...


# **TF-IDF**

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_sample['lematized'])
tfidf_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [36]:
print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")

TF-IDF Matrix Shape: (4915, 33763)


In [37]:
tfidf_csvector = pd.DataFrame(tfidf_similarity, index=df_sample['name'], columns=df_sample['name'])
tfidf_csvector

name,Always The Same Blue Sky...,Card of spirits,The Lonesome Fog,Sad Robot Lilly,Pill Puzzle,Lux Alliance,Spacetours VR - Ep1 The Solar System,My Personal World,AVOPUG SHOW,Misfire,...,Starshot: Space Circus Fever,YiShi,Blade Prince Academy,Drive Thru Miami,Aegis,Lumen: World of Color,Triptych Book I: Blockhead,Cloud Jump,Hidden Object - Sweet Home,Zombie Rooms
name,,,,,,,,,,,,,,,,,,,,,
Always The Same Blue Sky...,1.000000,0.014824,0.019939,0.008087,0.033803,0.004790,0.026614,0.008262,0.035635,0.015528,...,0.008362,0.005517,0.034058,0.021799,0.013357,0.019638,0.014423,0.053631,0.035732,0.016724
Card of spirits,0.014824,1.000000,0.003464,0.010388,0.024095,0.050726,0.011043,0.012018,0.026982,0.029025,...,0.006440,0.029241,0.016134,0.041155,0.002974,0.018562,0.013012,0.013655,0.016894,0.017566
The Lonesome Fog,0.019939,0.003464,1.000000,0.006738,0.005254,0.018302,0.046340,0.011957,0.046297,0.022232,...,0.003745,0.002093,0.013962,0.010043,0.000804,0.016514,0.026240,0.010204,0.011023,0.024736
Sad Robot Lilly,0.008087,0.010388,0.006738,1.000000,0.005153,0.011478,0.008240,0.010219,0.018635,0.021307,...,0.016258,0.023765,0.041639,0.007307,0.021493,0.029113,0.040793,0.019898,0.067649,0.001330
Pill Puzzle,0.033803,0.024095,0.005254,0.005153,1.000000,0.015975,0.026957,0.016806,0.016816,0.015252,...,0.005399,0.015687,0.038273,0.057135,0.039573,0.030751,0.024063,0.008482,0.044165,0.038722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lumen: World of Color,0.019638,0.018562,0.016514,0.029113,0.030751,0.017508,0.018676,0.022106,0.062462,0.048676,...,0.011320,0.024721,0.032471,0.010499,0.019859,1.000000,0.048231,0.009771,0.039847,0.017321
Triptych Book I: Blockhead,0.014423,0.013012,0.026240,0.040793,0.024063,0.072451,0.027576,0.057783,0.028792,0.021377,...,0.019232,0.026239,0.042559,0.021692,0.047093,0.048231,1.000000,0.033154,0.054832,0.015769
Cloud Jump,0.053631,0.013655,0.010204,0.019898,0.008482,0.006047,0.038544,0.016577,0.002052,0.051956,...,0.024334,0.013670,0.027553,0.025097,0.022141,0.009771,0.033154,1.000000,0.008959,0.039106


In [38]:
def rec_tfidf(game_name, top=10):
    if game_name not in tfidf_csvector.index:
        print(f"Game '{game_name}' tidak ditemukan dalam TF-IDF matrix.")
        return pd.DataFrame()

    similarity_scores = tfidf_csvector.loc[game_name]
    sorted_scores = similarity_scores.sort_values(ascending=False)
    sorted_scores = sorted_scores.drop(labels=game_name, errors='ignore')

    # Ambil top N game paling mirip
    top_game_names = sorted_scores.head(top).index
    top_scores = sorted_scores.head(top).values
    recommended = df_sample[df_sample['name'].isin(top_game_names)][['name', 'release_year', 'content']].copy()

    # Tambahkan kolom skor similarity berdasarkan urutan yang benar
    recommended['similarity_score'] = recommended['name'].apply(lambda x: similarity_scores[x])
    recommended = recommended.set_index('name').loc[top_game_names].reset_index()

    print(f"Top {top} recommendations for '{game_name}':")
    # for i, row in recommended.iterrows():
    #     print(f"{i+1}. {row['name']} ({row['release_year']})")
    #     print(f"   Content: {row['content'][:100]}...")
    #     print(f"   Similarity Score: {row['similarity_score']:.4f}\n")

    return recommended

In [39]:
rec_tfidf('Cloud Jump', 5)

Top 5 recommendations for 'Cloud Jump':


,name,release_year,content,similarity_score
0,Nubarron: The adventure of an unlucky gnome,2020,a gorgeous digital painted fantasy world an am...,0.213529
1,Sky Conquest,2024,join our discord about the game ⭐ sky conquest...,0.194537
2,SLIDE: platformer,2015,in some platformers you die a lot. not because...,0.187508
3,Knight Fall,2024,'welcome to knight fall: survive bosses and cl...,0.175403
4,Jumps,2017,"jumps is a fast paced first person platformer,...",0.174998


# **Word2Vec**

In [40]:
# !pip install numpy==1.23.5

In [41]:
# !pip install gensim==4.3.1 --no-build-isolation

In [42]:
# from gensim.models import Word2Vec
# from gensim.utils import simple_preprocess
# import gensim.downloader as api